In [ ]:
import numpy as np
import xarray as xr
import multiprocessing as mp

In [ ]:
path = "/path/to/model/output/" 

# Helmholtz decomposition
We want to perform a Helmholtz decomposition of $v'T'$ into its non-divergent and non-rotational parts ([](Marshall1981), [](Jayne2002)). The non-rotational part of $\overrightarrow{\mathbf{v}'T'}$ can be represented by a potential $\phi$ with 
\begin{equation}
\nabla^{2}\phi = \nabla\cdot(\overrightarrow{\mathbf{v}'T'}),
\end{equation}
such that
\begin{equation}
\label{eq:helm}
\begin{pmatrix}(u'T')_{div} \\ (v'T')_{div}\end{pmatrix} = \begin{pmatrix} \frac{\partial\phi}{\partial x} \\ \frac{\partial\phi}{\partial y}\end{pmatrix}.
\end{equation}

This equation is solved for $\phi$ using Successive Over-Relaxation (SOR) with Neumann boundary conditions on the northern and southern boundaries (there are no boundary conditions at the western and eastern boundaries as the model is zonally periodic).


## Compile the Fortran functions
SOR is an iterative process and it is thus unavoidable to use loops. The main iterative loops are written in Fortran because they would be extremely slow in python. We compile them with `f2py` to use them within python.

In [ ]:
!source .venv/py_3/bin/activate; python3 -m numpy.f2py -c solve_poisson_SOR_black_NM.f95 -m solve_poisson_SOR_black_NM

In [ ]:
!source .venv/py_3/bin/activate; python3 -m numpy.f2py -c solve_poisson_SOR_red_NM.f95 -m solve_poisson_SOR_red_NM

Import the compiled functions

In [ ]:
import solve_poisson_SOR_black_NM
import solve_poisson_SOR_red_NM

## Define some functions

In [ ]:
# makegrid gives us some info about the grid that we need for the solver
def makegrid(data_in, data_dx):
    nx = data_in.shape[1]
    ny = data_in.shape[0]
    dx = data_dx
    dy = dx
    return (nx, ny, dx, dy)

# set_bnd_phi sets the Neumann boundary conditions at the northern and southern boundary
def set_bnd_phi(data):
    data[0, :] = data[1, :]
    data[-1, :] = data[-2, :]
    return data

The function `apply_SORphi` is the main function that applies SOR to solve for $\phi$ at one time `t` and one depth level `k`. 

In [ ]:
def apply_SORphi(t):
    # set desired residual error
    residual = 1e-6
    # set maximum allowed number of iterations
    max_iter= 1e4
    # extract the required time and depth from data and conver to Fortran order
    npData = np.array(D[t, k, firstwet-1::, :], order='F')
    if k == 0:
        initial = init
    else:
        initial = init[t, :, :].squeeze()
    # extend data in y direction to account for the boundaries
    extend = np.zeros(npData.shape[1])
    npData = np.vstack((npData, extend))
    nx, ny, dx, dy = makegrid(npData, 10000.)
    # set initial guess
    if np.all(init == None):
        data_out = np.zeros((ny, nx), order='F')
    else:
        data_out = np.array(initial, order='F')
    # set omega (needs to be tuned depending on field to be solved)
    om = 1.99
    it = 1
    res = 1
    # iterate as long as the residual error is larger than `residual`
    while( res  >= residual ):
        pn = data_out.copy()
        solve_poisson_SOR_black_NM.solve(npData, data_out, dx, dy, om, int(nx), int(ny))
        solve_poisson_SOR_red_NM.solve(npData, data_out, dx, dy, om, int(nx), int(ny))
        data_out = set_bnd_phi(data_out)
        if pn.sum() == 0:
            res = 1
        else:
            res = abs((pn.sum() - data_out.sum()) / pn.sum())
        if it < max_iter:
            it += 1
        else:
            break
    return data_out

## Loop over all times and depths
Now we loop over all years and depth levels to perform the Helmholtz decomposition.

In [ ]:
# loop over the 10 decades
for y in range(0, 10):
    print(y)
    y1 = "02" + str(y) + "1"
    y2 = "0" + str(int(y1)+9)
    # loop over each year in decade
    for Y in range(int(y1), int(y2) + 1):
        # load THT
        THT = xr.open_mfdataset(path + "post/THT." 
                                + y1 + "_" + y2 + ".nc").sel(time=slice("0" + str(Y) + "-01-01",
                                                                        "0" + str(Y) + "-12-30"))
        # do some extension do account for boundaries and set land values to NaN
        Utmp = THT.THTu
        U = np.zeros((np.shape(Utmp)[0], np.shape(Utmp)[1], np.shape(Utmp)[2] + 1, np.shape(Utmp)[3]))
        U[:, :, 0:-1, :] = Utmp
        for k in np.arange(0, np.shape(U)[1]):
            firstwet = np.argmax(U[0, k, :, 0] != 0)
            U[:, k, 0:firstwet, :] = np.nan
        V = np.zeros(np.shape(U))
        V[:, :, 0:-1, :] = THT.THTv
        for k in np.arange(0, np.shape(V)[1]):
            firstwet = np.argmax(V[0, k, :, 0] != 0)
            V[:, k, 0:firstwet-1, :] = np.nan
        # compute d(THT_u)/dx and d(THT_v)/dy
        dUx = np.zeros(U.shape)
        dUx[:, :, :, 0:-1] = (U[:, :, :, 1::] - U[:, :, :, 0:-1]) / 10000.
        dUx[:, :, :, -1] = (U[:, :, :, 0] - U[:, :, :, -1]) / 10000.
        dVy = np.zeros(U.shape)
        dVy[:, :, 0:-1, :] = (V[:, :, 1::, :] - V[:, :, 0:-1, :]) / 10000.
        dVy[:, :, -1, :] = (V[:, :, 0, :] - V[:, :, -1, :]) / 10000.
        D = (dUx + dVy)[:, :, 0:-1, :]
        D[np.isnan(D)] = 0
        # initialize phi
        phi = np.zeros((np.shape(D)[0], np.shape(D)[1],
                        np.shape(D)[2]+1, np.shape(D)[3]))
        # loop over each depth level
        for k in np.arange(0, np.shape(D)[1]):
            print("working on depth level", k)
            firstwet = np.argmax(D[0, k, :, 0] != 0)
            if k == 0:
                init = None
            else:
                init = phi[:, k-1, firstwet-1::, :].squeeze()
            # perform SOR on as many cpus in parallel as possible
            # (we leave 2 cpus for other things)
            if __name__ == "__main__":
                with mp.Pool(mp.cpu_count() - 2) as p:
                    phi[:, k, firstwet-1::, :] = p.map(apply_SORphiNew, np.arange(0, np.shape(D)[0]))
                p.close()
                p.join()
        phi[phi==0] = np.nan
        # dphi/dy gives meridional component
        vdiv = np.zeros(np.shape(D))
        vdiv[:, :, 1::, :] = (phi[:, :, 1:-1, :] - phi[:, :, 0:-2, :]) / 10000.
        ds_vdiv = xr.Dataset({"time": (["time",], THT.time.data),
                              "Z": (["Z",], THT.Z.data),
                              "YG": (["YG",], THT.YG.data),
                              "XC": (["XC",], THT.XC.data)})
        ds_vdiv["THTvdiv"] = xr.DataArray(vdiv, dims=["time", "Z", "YG", "XC"])
        ds_vdiv.to_netcdf(path + "post/THTvdiv.0" + str(Y) + ".nc")